<a href="https://colab.research.google.com/github/luisanovaes/GitHub/blob/master/Classificador_BERT_Portugues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar as bibliotecas necessárias para utilizar a rede transformers

In [ ]:
!pip install simpletransformers

Autorizar acesso ao google drive para acessar base de dados e salvar outputs

In [4]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Mounted at /content/drive


Importar base de dados do sheets e transformar em dataframe; 

In [7]:
import pandas as pd
import matplotlib.pyplot as plt

##importando a tabela
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eS2a9h676hpFhQsisSdJCgdqYaDpReyMgL-eREZ4-XQ/edit#gid=1423440435')

#Selecionando os dados de cada aba
sheet1 = wb.worksheet('Treino')
sheet2 = wb.worksheet('Teste')

#Selecionando os dados e passando para um dataframe
data1 = sheet1.get_all_values()
data2 = sheet2.get_all_values()

#Transformando a coluna "Classificação" para o dtype inteiro (int) para que a rede aceite os dados como entrada
train_df = pd.DataFrame(data1)
train_df.columns = ['query','classificacao']
train_df['classificacao'] = train_df['classificacao'].astype(int)


test_df = pd.DataFrame(data2)
test_df.columns = ['query','classificacao']
test_df['classificacao'] = test_df['classificacao'].astype(int)


#vizualizar dados
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   query          361 non-null    object
 1   classificacao  361 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


Importar as bibliotecas, criar o classificador, passar os dados de treino e depois de teste para avaliar o modelo:

In [ ]:
# Classificador

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#train_data = train_df

# Preparing eval data
#eval_data = test_df


# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)
model_args.overwrite_output_dir = True

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'neuralmind/bert-base-portuguese-cased',
    num_labels=3,
    use_cuda=False,
    args=model_args,   
) 

# Train the model
model.train_model(train_df)
model_args.overwrite_output_dir =True


# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)




Fazer predições com o modelo e imprimir resultados. Criar um arquivo CVS com os outputs da rede no teste.

In [14]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["Petição Inicial", 'Lei Seca', 'Artigo 156 da cf', 'luisa p novaes', 'modelo de contrato para caso previsto em lei'])

print(result,'\n')
print(model_outputs, '\n')
print(wrong_predictions)
print(predictions, '\n', raw_outputs)


df_output = pd.DataFrame(model_outputs)
df_output.to_csv('drive/MyDrive/Results-BERT/output.csv')




INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'mcc': 0.7834474409115436, 'eval_loss': 0.2712684310770229} 

[[-2.21510124  3.70053434 -1.69147921]
 [-2.23462677  3.72634816 -1.64966142]
 [-2.33299947  3.9861443  -1.78036296]
 ...
 [ 1.51363826 -2.79395843  1.38060868]
 [ 1.58448505 -3.51427794  1.9615649 ]
 [ 2.14164424 -1.44846547 -0.42593533]] 

[]
[2 1 1 0 1] 
 [[ 0.14318877 -3.17952418  2.99025631]
 [-2.2293992   3.83150887 -1.66308057]
 [-2.01833892  3.46450353 -1.27464783]
 [ 2.61239576 -1.68504226 -0.65745515]
 [-1.93260705  3.03315783 -1.44880974]]
